In [5]:
import FaaSET
import FaaSETUI
import FaaSRunner

In [3]:
function_config = {
    "role": "arn:aws:iam::616835888336:role/service-role/simple_microservice_role",
    "memory": 256,
    "profile": "personal"
}

# Create a Function

Simply write your function in the Jupyter Notebook and it will automatically be deployed when ran. This example highlights using a variety of our tools including SAAF and FaaSETUI.

In [ ]:
@FaaSET.cloud_function(platform="aws", config=function_config)
def dynamic_function2(request, context): 
    from SAAF import Inspector
    import base64
    inspector = Inspector()  
    inspector.inspectAll()
    try:  
        encoded_code = request['f']
        if 'd' in request:
            import zipfile
            import sys
            encoded_dependencies = request['d']
            decoded_data = base64.b64decode(encoded_dependencies)
            with open('/tmp/dependencies.zip', 'wb') as f:
                f.write(decoded_data)
            with zipfile.ZipFile('/tmp/dependencies.zip', 'r') as zip_ref:
                zip_ref.extractall('/tmp')
            sys.path.append('/tmp')
            
        source = base64.b64decode(encoded_code).decode('utf-8')
        name = source.split("\n")[0].replace("def ", "").replace("(request, inspector):", "").strip()
        inspector.addAttribute("functionName", name) 
        exec(source + "\n" + 'inspector.addAttribute("return",' + name + "(" + str(request) + ", inspector))")
    except Exception as e:   
        inspector.addAttribute("dynamic_function_error", str(e))
    inspector.inspectAllDeltas() 
    return inspector.finish() 

FaaSETUI.UI(dynamic_function2)

dynamic_function2({"name": "Steve"})["message"]

Open: ./functions/hello_world/aws-us-east-2/default_config.json


In [ ]:
def hello_world(request, inspector):
    inspector.addAttribute("message", "Hello, World!")
    return None

dynamic_function2(FaaSET.dynamic_get_payload(hello_world, {}))